In [327]:
# dependencies
import pandas as pd
import numpy as np

pd.pandas.set_option('display.max_columns',None)

In [328]:
# variables
ARTWORKS_DATA_PATH="./data/Artworks.csv"

In [329]:
# read dataset
data = pd.read_csv(ARTWORKS_DATA_PATH, delimiter=",")

In [330]:
df = data.copy()
df.head(3)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjUyNzc3MCJd...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,Y,3,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjUyNzM3NCJd...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,Y,4,http://www.moma.org/collection/works/4,http://www.moma.org/media/W1siZiIsIjUyNzM3NSJd...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN


In [331]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140848 entries, 0 to 140847
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Title               140800 non-null  object 
 1   Artist              139632 non-null  object 
 2   ConstituentID       139632 non-null  object 
 3   ArtistBio           135023 non-null  object 
 4   Nationality         139632 non-null  object 
 5   BeginDate           139632 non-null  object 
 6   EndDate             139632 non-null  object 
 7   Gender              139632 non-null  object 
 8   Date                138743 non-null  object 
 9   Medium              131217 non-null  object 
 10  Dimensions          131596 non-null  object 
 11  CreditLine          138985 non-null  object 
 12  AccessionNumber     140848 non-null  object 
 13  Classification      140847 non-null  object 
 14  Department          140848 non-null  object 
 15  DateAcquired        134166 non-nul

### Workflow

-Fromat columns & remove redudndant `()`
      - `ArtistBio`
      - `Nationality`
      - `BeginDate`
      - `EndDate`
      - `Gender`


- split `Title` column to get only Relevant title

- Drop `Dimensions` column   
  - There are `lenth` and `width` columns with the same data already 
  - Resolve to use cm as a central metric for measurement


- handle missning/Null values

- Infer data-types

- Save as `parquet` file to retain infered data types


In [332]:
# format columns with redudant brackets
columns_w_redundant_brackets=["ArtistBio","Nationality","BeginDate","EndDate","Gender"]

for column in columns_w_redundant_brackets:
    df[column] = df[column].str.replace(r'\(|\)', '', regex=True)

In [333]:
# split ArtistBio column to use only relevant info (Nationalty data only)
df['ArtistBio'] = df['ArtistBio'].str.split(',').str[0].str.strip()

# split Title column to use only relevant Title info
df['Title'] = df['Title'].str.split(',').str[0].str.strip()

- values in ArtistBio seem to exist already in `Nationality`, `BeginDate` & `EndDate` columns
- ArtistBio	& Nationality seem to have similar info. could be used to fill in missing values

In [334]:
# Drop Dimesions column since `lenth` and `width` columns with the same data already 
df = df.drop('Dimensions', axis=1)

In [335]:
# function to find percentage of missing values
def find_percentage_missing(data):
    # find columns with missing values
    column_na=[features for features in data.columns if data[features].isnull().sum()>1]

    result_list = []

    for feature in column_na:
        missing_percentage = np.round(data[feature].isnull().mean() * 100, 2)
        result_list.append({'Column Name': feature, 'Missing Values Percentage': missing_percentage})

    return pd.DataFrame(result_list)

find_percentage_missing(df)

,Column Name,Missing Values Percentage
0,Title,0.03
1,Artist,0.86
2,ConstituentID,0.86
3,ArtistBio,4.14
4,Nationality,0.86
5,BeginDate,0.86
6,EndDate,0.86
7,Gender,0.86
8,Date,1.49
9,Medium,6.84


In [336]:
# drop columns with high % of null values (% >= 85)
large_na_columns=["Circumference (cm)", 
                  "Depth (cm)", "Diameter (cm)", "Length (cm)", "Weight (kg)", "Seat Height (cm)", "Duration (sec.)"]
df = df.drop(columns=large_na_columns)

In [337]:
# fill title with "No Title" (% null values - < 1%)
df['Title'].fillna("No Title", inplace=True)

# Fill Artists column with Artist Unavailable (% null values - < 1%)
df['Artist'].fillna("Artist Unavailable", inplace=True)

# ID columns are very unique & cannot be randomly filled  (% null values - < 1%)
df.dropna(subset=['ConstituentID'], inplace=True)




# merge ArtistBio & Nationality
#  ArtistBio & Nationality have same data to merge - Taking Nationality as majority
df['merged_Nationality_ArtistBio'] = df['Nationality'].combine_first(df['ArtistBio'])

# drop null rows
df = df[pd.notna(df['merged_Nationality_ArtistBio'])]

# discrad Nationality & ArtistBio after merge
df = df.drop(columns=["Nationality", "ArtistBio"])
df.rename(columns={'merged_Nationality_ArtistBio': 'Nationality'}, inplace=True)

# fill more empty ' ' spaces
mode_nationality = df['Nationality'].mode().values[0]
df['Nationality'] = df['Nationality'].replace('', mode_nationality)

# clean multiple Nationality in one sample
split_df = df['Nationality'].str.split(expand=True)
df['Nationality'] = split_df[0]
df['Nationality'].fillna(mode_nationality, inplace=True)



# replace with mode of Medium
mode_medium = df['Medium'].mode().values[0]
df['Medium'] = df['Medium'].fillna(mode_medium)

# replace height and width with most occuring (use mode incase of outliers)
mode_height = df['Height (cm)'].mode().values[0]
df['Height (cm)'] = df['Height (cm)'].fillna(mode_height)

mode_width = df['Width (cm)'].mode().values[0]
df['Width (cm)'] = df['Width (cm)'].fillna(mode_width)


# replace url, thumbnail, creditline with not available (since URL must be unique)
df['URL'].fillna("Unavailable", inplace=True)
df['ThumbnailURL'].fillna("Unavailable", inplace=True)
df['CreditLine'].fillna("Unavailable", inplace=True)


# DateAcquired column (% null values - = 4%)
mode_date_acquired = df['DateAcquired'].mode().values[0]
df['DateAcquired'] = df['DateAcquired'].fillna(mode_date_acquired)

# unresolved 
df = df.drop(columns=["Date"])

# Classification column
mode_date_acquired = df['Classification'].mode().values[0]
df['Classification'] = df['Classification'].fillna(mode_date_acquired)

# gender handling
gender_mapping = {
    'male': 'Male',
    'female': 'Female',
    'non-binary': 'Non-Binary',
    '': "undisclosed" }

# clean inconsistent gender naming 
df['Cleaned_Gender'] = df['Gender'].str.lower().map(gender_mapping)

# fill null gender spaces 
df['Cleaned_Gender_nan'] = df['Cleaned_Gender'].fillna("undisclosed")

# drop after use 
df = df.drop(columns=["Gender","Cleaned_Gender"])
df.rename(columns={'Cleaned_Gender_nan': 'Gender'}, inplace=True)


In [338]:
df.head()

,Title,Artist,ConstituentID,BeginDate,EndDate,Medium,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Height (cm),Width (cm),Nationality,Gender
0,Ferdinandsbrücke Project,Otto Wagner,6210,1841,1918,Ink and cut-and-pasted painted pages on paper,Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjUyNzc3MCJd...,48.6000,168.9000,Austrian,Male
1,City of Music,Christian de Portzamparc,7470,1944,0,Paint and colored pencil on print,Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,Y,3,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjUyNzM3NCJd...,40.6401,29.8451,French,Male
2,Villa near Vienna Project,Emil Hoppe,7605,1876,1957,"Graphite, pen, color pencil, ink, and gouache ...",Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,Y,4,http://www.moma.org/collection/works/4,http://www.moma.org/media/W1siZiIsIjUyNzM3NSJd...,34.3000,31.8000,Austrian,Male
3,The Manhattan Transcripts Project,Bernard Tschumi,7056,1944,0,Photographic reproduction with colored synthet...,Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,Y,5,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjUyNzQ3NCJd...,50.8000,50.8000,American,Male
4,Villa,Emil Hoppe,7605,1876,1957,"Graphite, color pencil, ink, and gouache on tr...",Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,Y,6,http://www.moma.org/collection/works/6,http://www.moma.org/media/W1siZiIsIjUyNzQ3NSJd...,38.4000,19.1000,Austrian,Male


In [339]:
# % of null BeginDate & EndDate in columns
percentage_of_zeros_in_beginDate = (df['BeginDate'] == "0").mean() * 100 
percentage_of_zeros_in_beginDate = (df['EndDate'] == "0").mean() * 100

print(f"% of null values in dates -> \nBeginDate: {percentage_of_zeros_in_beginDate} \nEndDate: {percentage_of_zeros_in_beginDate}")

% of null values in dates -> 
BeginDate: 32.53480577518047 
EndDate: 32.53480577518047


In [340]:
df["BeginDate"].value_counts()

BeginDate
0                                                                8039
1857                                                             5111
1911                                                             3980
1886                                                             3805
1923                                                             3236
                                                                 ... 
1940 1953 1957 1953 1963 1960 1952 1954 1949 1954 1941 0            1
0 1930 1928 1929 1923 1924                                          1
1901 1930 1923 1912 1923 1896 1915 1929 1925 1914 1936 1928 0       1
1945 1939                                                           1
0 1939 1938 1930 1955 1970 1972 1962 1937 1938 1977 1969 1955       1
Name: count, Length: 2238, dtype: int64

In [341]:
df["EndDate"].value_counts()

EndDate
0                    45429
1927                  5138
2010                  4174
1985                  3417
1969                  3330
                     ...  
2011 0 2005 0 0 0        1
2005 0 2011 0 0          1
0 0 2011 2005 0          1
0 2011 0 2005 0          1
1979 1968                1
Name: count, Length: 1316, dtype: int64

In [342]:
# clean inconsistent BeginDate & EndDate formating
# replace sample with date length greater than standard year length with 0
df.loc[df['BeginDate'].str.len() > 4, 'BeginDate'] = "0"
df["BeginDate"].value_counts()

BeginDate
0       15627
1857     5111
1911     3980
1886     3805
1923     3236
        ...  
1789        1
1994        1
1787        1
1772        1
2016        1
Name: count, Length: 222, dtype: int64

In [343]:
df.loc[df['EndDate'].str.len() > 4, 'EndDate'] = "0"
df["EndDate"].value_counts()

BeginDate
0       15627
1857     5111
1911     3980
1886     3805
1923     3236
        ...  
1789        1
1994        1
1787        1
1772        1
2016        1
Name: count, Length: 222, dtype: int64

In [344]:
# % of null dates in columns
percentage_of_zeros_in_beginDate = (df['BeginDate'] == "0").mean() * 100 
percentage_of_zeros_in_beginDate = (df['EndDate'] == "0").mean() * 100

print(f"% of null values in dates -> \nBeginDate: {percentage_of_zeros_in_beginDate} \nEndDate: {percentage_of_zeros_in_beginDate}")

% of null values in dates -> 
BeginDate: 36.68643290936175 
EndDate: 36.68643290936175


In [345]:
# replace 0 values with mode of date column
df_w_no_zero_beginDate = df[df["BeginDate"] != "0"]
mode_date_beginDate = df_w_no_zero_beginDate["BeginDate"].mode().values[0]
df['BeginDate'] = df['BeginDate'].replace('0', mode_date_beginDate)
df['BeginDate'] = df['BeginDate'].replace("0 0", mode_date_beginDate)


In [346]:
df_w_no_zero_beginDate = df[df["EndDate"] != "0"]
mode_date_beginDate = df_w_no_zero_beginDate["EndDate"].mode().values[0]
df['EndDate'] = df['EndDate'].replace('0', mode_date_beginDate)
df['EndDate'] = df['EndDate'].replace("0 0", mode_date_beginDate)

In [347]:
# infer data types
df['BeginDate'] = pd.to_datetime(df['BeginDate'], format='%Y')
df['EndDate'] = pd.to_datetime(df['EndDate'], format='%Y')

In [348]:
# complete date format
df['DateAcquired'] = pd.to_datetime(df['DateAcquired'])

In [349]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 139632 entries, 0 to 140847
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Title            139632 non-null  object        
 1   Artist           139632 non-null  object        
 2   ConstituentID    139632 non-null  object        
 3   BeginDate        139632 non-null  datetime64[ns]
 4   EndDate          139632 non-null  datetime64[ns]
 5   Medium           139632 non-null  object        
 6   CreditLine       139632 non-null  object        
 7   AccessionNumber  139632 non-null  object        
 8   Classification   139632 non-null  object        
 9   Department       139632 non-null  object        
 10  DateAcquired     139632 non-null  datetime64[ns]
 11  Cataloged        139632 non-null  object        
 12  ObjectID         139632 non-null  int64         
 13  URL              139632 non-null  object        
 14  ThumbnailURL     139632 n

In [350]:
df.to_parquet('./data/cleaned_data.parquet', index=False)